In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm
import scipy.io as sio
from scipy import stats
from importlib import reload
from matplotlib import pyplot as plt
import sys
#sys.path.append('/home/jma819/post_cmfe_analysis')
sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import python_utils_jjm as utils_jjm
import dlc_utils
import caiman
import statsmodels.formula.api as smf
import scipy.spatial.distance as dist
import itertools
import math
import warnings
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
warnings.filterwarnings(action='once')

In [3]:
reload(utils_jjm)
reload(dlc_utils)

<module 'dlc_utils' from '/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/dlc_utils.py'>

In [4]:
cnmfe_file_key = pd.read_csv('/volumes/My_Passport/dlc_analysis/behavcamvideos/cnmfe_file_key.csv')
#cnmfe_file_key = pd.read_csv('/projects/p30771/dlc_analysis/openfield_dlc_output/cnmfe_file_key.csv')

In [5]:
cnmfe_base_dir = '/Volumes/My_Passport/cnmfe_analysis_files/batch_output_files/'
#cnmfe_base_dir = '/projects/p30771/MATLAB/CNMF_E_jjm/quest_MATLAB_cnmfe/batch_output_files/'
wt_CNMFE_file = ['30-Mar_20_39_05_out.mat', '30-Mar_20_45_16_out.mat', '27-Feb_17_33_59_out.mat', '22-Mar_22_52_02_out.mat',
                 '28-Feb_16_10_05_out.mat', '27-Feb_17_32_15_out.mat', '28-Feb_16_21_21_out.mat', '25-Mar_13_27_27_out.mat',
                 '25-Mar_14_22_02_out.mat', '25-Mar_14_22_44_out.mat', '26-Mar_18_33_55_out.mat', '27-Mar_00_26_12_out.mat', '27-Mar_00_48_46_out.mat']

ko_CNMFE_files = ['31-Mar_13_28_15_out.mat', '29-Mar_21_42_20_out.mat', '13-Apr_17_57_40_out.mat', '29-Mar_14_27_55_out.mat', '13-Apr_16_01_20_out.mat',
                 '13-Apr_16_11_27_out.mat', '29-Mar_13_39_44_out.mat']

In [6]:
tracking_files = ['/Volumes/My_Passport/dlc_analysis/behavcamvideos/open_field_miniscope/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in ko_CNMFE_files] 
#tracking_files = ['/projects/p30771/dlc_analysis/openfield_dlc_output/'+utils_jjm.find_behavior_tracking(fname, cnmfe_file_key) for fname in wt_CNMFE_file] 

In [7]:
CNMFE_dir_paths_list = [str(cnmfe_base_dir+fname) for fname in ko_CNMFE_files]
#CNMFE_dir_paths_list

In [8]:
binning_time = 1 # seconds 
body_part_for_tracking = 'tail_base' #
number_of_bins = 50 #
polynomial_degree = 2 #

grouped_raw_data = {}
success = []
failed = []
for CNMFE_file, tracking_file in zip(CNMFE_dir_paths_list, tracking_files):
    #print(CNMFE_file)
    #print(tracking_file)
    try:
        # load cell fluorescence 
        cell_fluorescence = sio.loadmat(CNMFE_file)
        C_timedelta = utils_jjm.create_fluorescence_time_delta(cell_fluorescence['C'])
        C_normalized = C_timedelta.apply(utils_jjm.normalize).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_z_scored = C_timedelta.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_timedelta)-1)*(1/20), len(C_timedelta)), unit='s'), drop=True)
        C_normalized_z_scored = C_normalized.apply(stats.zscore).set_index(pd.to_timedelta(np.linspace(0, (len(C_normalized)-1)*(1/20), len(C_normalized)), unit='s'), drop=True)
        # create tracking time deltas
        interpolated = utils_jjm.prepare_timedelta_dfs(tracking_file)
        #load spatial components by session
        com_df, spatial_components = utils_jjm.return_spatial_info(CNMFE_file, 0.6)
        cell_contours, for_dims = utils_jjm.create_contour_layouts(spatial_components)
        #C_z_scored_filtered = utils_jjm.filter_out_by_size(C_z_scored, cell_contours, for_dims, 0.6, 100)
        #store results 
        grouped_raw_data[tracking_file.split('/')[-2]] = {'C': C_timedelta, 'C_z_scored': C_z_scored, 'C_normalized': C_normalized, 'C_normalized_z_scored': C_normalized_z_scored, 
                                                          'interpolated' : interpolated, 'com' : com_df, 'spatial_components' : spatial_components, 'cell_contours': cell_contours,  
                                                         'for_dims' : for_dims}
        success.append((tracking_file.split('/')[-2], CNMFE_file.split('/')[-1]))
    except FileNotFoundError:
        failed.append(tracking_file)
    except OSError:
        failed.append(tracking_file)

## triggered averaging 

In [9]:
## combine dfs for each session to bin velocity values across dfs
#downsample
new_sampling_interval = .2
V_df = pd.concat([grouped_raw_data[session]['interpolated'].resample(str(new_sampling_interval)+'S').max() 
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))
#bin recording periods by mouse velocity 
all_sessions_v_bins = pd.cut(V_df['tail_base'], bins=50)

V_df['velocity_bins'] = all_sessions_v_bins

In [10]:
## filters out detected "cells" that are smaller than a given pixel threshold (i.e noise)
filtered_for_analysis = {}
for session in list(grouped_raw_data.keys()):
    filtered_for_analysis[session] = utils_jjm.filter_out_by_size(grouped_raw_data[session]['C_normalized_z_scored'], grouped_raw_data[session]['cell_contours'], 
                                                                  grouped_raw_data[session]['for_dims'], 0.6, 100)
C_df = pd.concat([filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))


In [29]:
to_save = V_df.rename(columns={'tail_base':'animal_velocity'})['animal_velocity']
to_save.to_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/KO_animal_velocity.csv')

In [32]:
#to_save['GRIN012_H16_M57_S23']

In [28]:
# should save fluorescence output here to pass along along with velocity data, could just pull out tail base values 
#C_df.head()
C_df.head()
C_df.to_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/KO_cells_f_traces.csv')

In [18]:
#bin by activity threshold (body_part, resting_time_threshold, active_time_threshold, crossing_threshold, resting_threshold, activity_threshold)
binned_velocity_df = pd.DataFrame([dlc_utils.bin_by_activity_threshold(V_df.loc[session]['tail_base'], 80, 20, 1, 2, 3)
                                for session in list(grouped_raw_data.keys())], index=list(grouped_raw_data.keys()))
binned_velocity_df = binned_velocity_df.transpose()

/Users/johnmarshall/anaconda3/envs/caiman/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/johnmarshall/anaconda3/envs/caiman/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [19]:
#binned_velocity_df

In [20]:
#plt.plot(binned_velocity_df['GRIN035_H14_M40_S34'])
#plt.show()

In [21]:
#binned_velocity_df

In [22]:
#fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharex=True)
#ax1.plot(binned_velocity_df['GRIN013_H13_M33_S54'].values)
#ax2.plot(V_df.loc['GRIN013_H13_M33_S54']['tail_base'].values)
#plt.show()

In [23]:
## get velocity threshold indicies and select fluorescence regions
triggered_activity_across_sessions = {}
triggered_activity_all_cells = {}
for session in list(grouped_raw_data.keys()):
    #inputs activity_threshold, resting_threshold, resting_baseline 
    crossing_indicies = utils_jjm.select_trigger_regions(binned_velocity_df[session], 0.5, 0.5, 20)
    #inputs time_to_plot
    threshold_activity = utils_jjm.average_triggered_regions(V_df.loc[session]['tail_base'].values, crossing_indicies, 80)
    cells_in_session = C_df.loc[session].dropna(axis=1)
    cell_fluorescence = cells_in_session.drop(['msCamFrame'], axis=1)
    f_threshold_activity = utils_jjm.average_triggered_regions(cell_fluorescence.mean(axis=1).values, crossing_indicies, 80)
    triggered_activity_across_sessions[session] = pd.concat([threshold_activity, f_threshold_activity], axis=1, keys=['velocity', 'fluorescence'])
    f_thresholds_cell = {}
    for cell in cell_fluorescence:
        f_thresholds_cell[cell_fluorescence[cell].name] = utils_jjm.average_triggered_regions(cell_fluorescence[cell].values, crossing_indicies, 80)
    triggered_activity_all_cells[session] = pd.concat(list(f_thresholds_cell.values()), axis=1, keys=list(f_thresholds_cell.keys()))

# all cells averaged together
triggered_activity_across_sessions_df = pd.concat(triggered_activity_across_sessions, axis=1)
# all cells individual
triggered_activity_all_cells_df = pd.concat(triggered_activity_all_cells, axis=1)

In [24]:
triggered_activity_all_cells_df.head()
triggered_activity_across_sessions_df.to_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/motion_events_velocity_wt.csv')

GRIN009_H13_M59_S14                                                    \
                   1                                                      
                 2111      3056      3182      4611      5030      5279   
0           -0.482954 -0.547562 -0.568077 -0.571950 -0.521606  2.147669   
1           -0.512150 -0.558620 -0.570167 -0.070401 -0.543488  2.440549   
2           -0.531865 -0.564844 -0.571343 -0.135690 -0.544573  2.110542   
3           -0.545177 -0.568347 -0.572006 -0.322338 -0.548248  2.344461   
4           -0.466662 -0.570319 -0.572378 -0.351388 -0.558755  1.926457   

                                           ... GRIN012_H16_M57_S23            \
         2                                 ...                  71             
       2111      3056      3182      4611  ...                2013      3163   
0 -0.815315 -0.228687  1.656980 -0.084804  ...           -0.468045 -0.286603   
1 -0.821856 -0.348074  1.165655 -0.148906  ...           -0.500051 -0.448076   
2 -0.826101 -0.443912  1.414289 -0.285969  ...           -0.526518 -0.065002   
3 -0.828857 -0.520846  1.475310  0.239970  ...           -0.548403 -0.144512   
4 -0.830646 -0.582605  1.899600 -0.065629  ...           -0.566500 -0.368613   

                                                                         \
         81                                      56                       
       381       1409      2013      3163      381       1409      2013   
0 -0.295413 -0.913994 -0.055518 -0.400989 -0.404955  0.369142 -0.082263   
1 -0.395563 -0.913994 -0.000928 -0.452729 -0.199797  0.351260 -0.122969   
2 -0.447242 -0.913994  0.009776 -0.369003 -0.325808  0.417354 -0.161053   
3 -0.486694 -0.913994  0.346042 -0.423969 -0.447182  0.625306 -0.196684   
4 -0.187210 -0.913994  0.051542 -0.488732 -0.530638  0.539032 -0.160429   

             
             
       3163  
0 -0.459099  
1 -0.482723  
2 -0.504161  
3 -0.523614  
4 -0.541266  

[5 rows x 2849 columns]

In [27]:
list(set([column[0] for column in triggered_activity_all_cells_df.columns]))

['GRIN009_H13_M59_S14',
 'GRIN012_H17_M32_S17',
 'GRIN011_H10_M19_S59',
 'GRIN012_H16_M57_S23']

## display averaged data

In [ ]:
# create range scaled data to compare across cells
scaler = MinMaxScaler()
activity_scaled = pd.DataFrame(scaler.fit_transform(triggered_activity_across_sessions_df), columns=triggered_activity_across_sessions_df.columns)

# display individual cell data 

In [ ]:
# create range scaled data to compare across cells
scaler = MinMaxScaler()
activity_scaled = pd.DataFrame(scaler.fit_transform(triggered_activity_all_cells_df), columns=triggered_activity_all_cells_df.columns)
activity_scaled_cell_means = activity_scaled.groupby(level=[0,1], axis=1).mean()

In [ ]:
plt.plot(activity_scaled_cell_means['GRIN009_H13_M59_S14'][1])
plt.show()

In [ ]:
activity_scaled_cell_means.head()

In [ ]:
# now i want to find the latency from the threshold crossing and sort the events in that manner 

In [ ]:
threshold_pt = activity_scaled_cell_means.shape[0]/2

In [ ]:
latency = activity_scaled_cell_means.idxmax(axis=0)
latency_flat = latency.reset_index()
latency_sorted = latency_flat.sort_values(by=0)

In [ ]:
sorted_heatmap = np.empty([activity_scaled_cell_means.shape[1], activity_scaled_cell_means.shape[0]])

In [ ]:
#activity_scaled_cell_means['GRIN035_H14_M40_S34'][49]

In [ ]:
for row_idx in range(len(list(latency_sorted.index))):
    sorted_heatmap[row_idx] = activity_scaled_cell_means[latency_sorted.loc[list(latency_sorted.index)[row_idx]]['level_0']][latency_sorted.loc[list(latency_sorted.index)[row_idx]]['level_1']]
    

In [ ]:
# heatmap

#fluorescence_events = np.transpose(sorted_heatmap)
X_labels = np.linspace(-16, 16, 161)
ax = sns.heatmap(sorted_heatmap, xticklabels=X_labels)
ax.set_xticks(ax.get_xticks()[::20])
ax.set_xticklabels(X_labels[::20])
fig = plt.gcf()
fig.set_size_inches(10, 6)
ax = plt.gca()
ax.set_ylabel('cell average F/F0 scaled')
ax.set_xlabel('latency from movement onset (sec)')
ax.axvline(x=(80), linestyle='--', lw=3, color='k')
#fig.subplots_adjust(left=0.15)
fig.savefig('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/motion_onset_alignment/cell_avg_aligned_heatmap_KO.pdf', transparent=True)
plt.show()


In [ ]:
## plot a histogram of the latency to threshold
plt.hist(latency_sorted[0], 50)
plt.show()


In [ ]:
cumulative_latency = stats.cumfreq(latency_sorted[0].values, numbins=100, defaultreallimits=(0, 160))

In [ ]:
cumulative_data = cumulative_latency.cumcount/len(latency_sorted[0].values)

In [ ]:
plt.rcParams.update({'font.size': 22})
plt.plot(np.linspace(0, 160, 100), cumulative_data, linewidth=7.0)
#plt.plot(np.linspace(0, 5, 100), ko_data, color='darkseagreen', linewidth=7.0)
plt.xlim((0, 160))
fig = plt.gcf()
fig.set_size_inches(10, 6)
ax = plt.gca()
ax.set_ylabel('cumulative fraction')
ax.set_xlabel('latency from movement onset (samples)')
fig.subplots_adjust(left=0.15)
#fig.savefig('/home/jma819/presentation_files/df_binned_by_v.pdf', transparent=True)
plt.show()

In [ ]:
# save data to plot next to wt
latency_sorted[0].to_csv('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/motion_onset_alignment/latency_to_movement_onset_KO.csv')

In [ ]:
##average together by session 

In [ ]:
#concat by mouse 
concacted_by_mouse = {}
for mouse in list(set([session[0:7] for session in list(triggered_activity_across_sessions.keys())])):
    dfs_by_mouse = []
    sessions = []
    for session in list(triggered_activity_across_sessions.keys()):
        if mouse in session:
            if not(triggered_activity_across_sessions[session].empty):
                means = triggered_activity_across_sessions[session].mean(axis=1, level=0)
                dfs_by_mouse.append(means)
                sessions.append(session)
    if len(dfs_by_mouse)>0:
        concacted_by_mouse[mouse] = pd.concat(dfs_by_mouse, axis=1, keys=sessions)
combined_by_mouse = pd.concat(list(concacted_by_mouse.values()), axis=1, keys=list(concacted_by_mouse.keys()))

In [ ]:
combined_by_mouse_timedelta = combined_by_mouse.set_index(pd.to_timedelta(np.linspace(0, (len(combined_by_mouse)-1)*(1/5), len(combined_by_mouse)), unit='s'), drop=True)
resampled_for_plotting = combined_by_mouse_timedelta.resample('.4S').mean()

In [ ]:
#resampled_for_plotting.head()

In [ ]:
plt.rcParams.update({'font.size': 22})
df_to_plot = resampled_for_plotting
activity_measure = 'spatial_coordination'
x = np.linspace(-(len(df_to_plot)/2)*.4, (len(df_to_plot)/2)*.4, len(df_to_plot))
scale_f = 0.0868
mean=df_to_plot.mean(axis=1, level=2)['velocity'].values*scale_f
plt.plot(x, df_to_plot.mean(axis=1, level=2)['velocity'].values*scale_f, color='k')
std_error = ((df_to_plot.std(axis=1, level=2)['velocity'])/math.sqrt(df_to_plot.mean(axis=1, level=0).shape[1]))*scale_f
plt.fill_between(x, mean-std_error, mean+std_error, color='g')
ax = plt.gca()
ax.set_ylabel('velocity cm/sec')
ax.set_xlabel('time (seconds)')
ax.axvline(x=(0), linestyle='--', color='r')
fig = plt.gcf()
fig.set_size_inches(7, 6)
#fig.draw()
#fig.savefig('/home/jma819/presentation_files/wt_velocity_spatial_coord_triggered.svg', transparent=True)
fig.subplots_adjust(left=0.15)
fig.savefig('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/motion_onset_alignment/v_triggered_KO.pdf', transparent=True)
plt.show()

In [ ]:
activity_measure = 'fluorescence'
x = np.linspace(-(len(df_to_plot)/2)*.4, (len(df_to_plot)/2)*.4, len(df_to_plot))
mean = df_to_plot.mean(axis=1, level=2)[activity_measure].values
plt.plot(x, mean, color='k')
std_error = (df_to_plot.std(axis=1, level=2)[activity_measure])/math.sqrt(df_to_plot.mean(axis=1, level=0).shape[1]*1)
plt.fill_between(x, mean-std_error, mean+std_error, color='g')
ax = plt.gca()
ax.set_ylabel('dF/F z-score')
ax.set_xlabel('time (seconds)')
ax.set_ylim([-0.02, 1.2])
ax.axvline(x=0, linestyle='--', color='r')
fig = plt.gcf()
fig.set_size_inches(7, 6)
#fig.draw()
#fig.savefig('.svg')
fig.subplots_adjust(left=0.15)
fig.savefig('/Volumes/My_Passport/cnmfe_analysis_files/OpenFieldAnalysis/2020/motion_onset_alignment/fluorescence_triggered_KO.pdf', transparent=True)
plt.show()

## event ISIs 

In [ ]:
filtered_for_analysis = {}
for session in list(grouped_raw_data.keys()):
    filtered_for_analysis[session] = utils_jjm.filter_out_by_size(grouped_raw_data[session]['C_normalized_z_scored'], grouped_raw_data[session]['cell_contours'], 
                                                                  grouped_raw_data[session]['for_dims'], 0.6, 100)
C_df = pd.concat([filtered_for_analysis[session].resample(str(new_sampling_interval)+'S').max()
                  for session in list(grouped_raw_data.keys())], keys=list(grouped_raw_data.keys()))

In [ ]:
C_df = C_df.drop(['msCamFrame'], axis=1)

In [ ]:
ISIs_by_session = []
for session in list(grouped_raw_data.keys()):
    print(session)
    event_indicies, event_times, event_ISIs = utils_jjm.get_ISIs(C_df.loc[session].dropna(axis=1), 20, .1, np.array(C_df.loc[session].dropna(axis=1).columns), 2)
    ISIs_by_session.append(event_ISIs)